In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

## 一、数据准备

In [2]:
train_data=datasets.FashionMNIST(
    root='data',#表示数据存储路径
    train=True,
    download=True,#若root中不包含数据，则从网络下载
    transform=ToTensor()#表示将特征转换为tensors
)

test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

/Users/singal/miniforge3/envs/py38/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-4gogm3_o/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

提取数据之后，需要分批次地将数据传入模型中，以降低过拟合，DataLoader能够满足我们的需求。

In [4]:
from torch.utils.data import DataLoader

train_dataloader=DataLoader(train_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64,shuffle=True)

In [5]:
train_dataloader.__dict__

{'dataset': Dataset FashionMNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 'num_workers': 0,
 'prefetch_factor': 2,
 'pin_memory': False,
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 64,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.RandomSampler at 0x13f43dac0>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x13f43d3a0>,
 'generator': None,
 'collate_fn': <function torch.utils.data._utils.collate.default_collate(batch)>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

In [9]:
next(iter(train_dataloader))

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0039, 0.0000,  ..., 0.0118, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

## 二、探究dataloader到底是什么
dataloader中共有（数据总数/batch_size）个样本，每个样本含有batch_size个数据。

dataloader.dataset中存储的是dataloader所使用的数据集，本段代码中即为上文的train_data。

In [6]:
train_features,train_labels=next(iter(train_dataloader))#next与iter搭配使用
train_features.size()#64对应上文的一次取64个样本，可视为样本的序号？，1、28、28为图像的特征

torch.Size([64, 1, 28, 28])

In [7]:
x,y=next(iter(train_dataloader))

In [8]:
x.size(),y.size()#每个样本含有batch_size个数据

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [9]:
x,y=next(iter(train_dataloader.dataset))#dataloader.dataset中存储的是dataloader所使用的数据集，本段代码中即为上文的train_data
x.size(),y

(torch.Size([1, 28, 28]), 9)

In [10]:
len(train_dataloader),len(train_dataloader.dataset)

(938, 60000)

## 二、数据转换
数据通常需要经过一系列处理，才能作为能被机器学习模型识别的数据，从而传入模型中，这就是transform的用武之地。为了训练模型（估计参数），特征数据需被转换为标准张量，标签也需要被转换为One-Hot编码，实现上述功能可以借助ToTensor和Lambda。

In [11]:
from torchvision.transforms import ToTensor,Lambda

In [12]:
ds=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y:torch.size(10).scatter_(dim=0,index=torch.tensor(y),value=1))
)

## 三、构造神经网络模型
### （一）概述
神经网络由多个层（layers），又名模块（modules）组成。torch.nn作为一个命名空间，提供多样化的“砖石”来帮助使用者构建神经网络。每一个模块均是nn.Moduel的子类。一个神经网络本身就是一个模块，且由众多子模块构成，即“嵌入式”结构——nested structure。

In [13]:
import os 
from torch import nn

In [14]:
device='cuda' if torch.cuda.is_available() else 'cpu'#截止2021年11月19日，m1芯片仍不支持cuda
print(device)

cpu


通过继承nn.Module来定义神经网络，在__init__中初始化神经网络层（layers）。每一个nn.Module的子类均使用forward方法对输入数据进行操作。

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()#PyTorch的指定语句
        #以下代码用来定义层：
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),#特征维度为28*28，512代表神经元个数
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)#10代表输出数据的维度，因为MNIST数据集的标签种类为10个
        )
    def forward(self,x):#用以对输入数据进行操作
        x=self.flatten(x)#操作之前，需将数据“变平”
        logits=self.linear_relu_stack(x)
        return logits

In [16]:
model=NeuralNetwork().to(device)#字面理解，将实例化后的对象“送入”（to）device中进行操作
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [17]:
X=torch.rand(1,28,28)#0-1均匀分布
logits=model(X)
logits

tensor([[-0.0049, -0.0056,  0.0767,  0.0209, -0.0046,  0.0901, -0.0629,  0.1156,
          0.0387, -0.1008]], grad_fn=<AddmmBackward0>)

最后一个线性组合层根据逻辑损失函数，返回了负值。将该负值传入nn.Softmax的某个实例中，即被重新转化为（scale）在[0,1]上的取值。dim参数表示沿着哪条轴/维度的数据之和为1。

In [18]:
pred_pro=nn.Softmax(dim=1)(logits)
pred_pro

tensor([[0.0977, 0.0976, 0.1060, 0.1003, 0.0977, 0.1074, 0.0922, 0.1102, 0.1021,
         0.0888]], grad_fn=<SoftmaxBackward0>)

In [19]:
y_pred=pred_pro.argmax(1)
y_pred

tensor([7])

In [20]:
for name, param in model.named_parameters():
    print(f'layer:{name} | Size:{param.size()} | \nValues:{param[:2]}\n')

layer:linear_relu_stack.0.weight | Size:torch.Size([512, 784]) | 
Values:tensor([[ 3.2909e-02,  2.6293e-02,  2.0720e-03,  ...,  2.4925e-02,
         -5.3411e-04, -1.5061e-05],
        [ 1.0271e-03,  2.0648e-02, -3.1945e-02,  ...,  1.6777e-02,
         -1.1508e-02,  1.3032e-02]], grad_fn=<SliceBackward0>)

layer:linear_relu_stack.0.bias | Size:torch.Size([512]) | 
Values:tensor([-0.0252,  0.0019], grad_fn=<SliceBackward0>)

layer:linear_relu_stack.2.weight | Size:torch.Size([512, 512]) | 
Values:tensor([[ 0.0262,  0.0050, -0.0305,  ..., -0.0315, -0.0395, -0.0084],
        [-0.0329,  0.0189, -0.0058,  ..., -0.0358,  0.0282, -0.0030]],
       grad_fn=<SliceBackward0>)

layer:linear_relu_stack.2.bias | Size:torch.Size([512]) | 
Values:tensor([-0.0368,  0.0178], grad_fn=<SliceBackward0>)

layer:linear_relu_stack.4.weight | Size:torch.Size([10, 512]) | 
Values:tensor([[-0.0228,  0.0092, -0.0194,  ...,  0.0416, -0.0203,  0.0415],
        [-0.0191, -0.0406, -0.0164,  ..., -0.0067,  0.0211, -0.

### （二）模块中的各种层layers

In [21]:
input_image=torch.rand(3,28,28)#生成3个28*28的图片数据作为演示

#### nn.Flatten
将每个样本的所有特征“铺平”，如某个样本是10*10维矩阵，则转换为100维向量。

In [22]:
flatten=nn.Flatten()
flat_image=flatten(input_image)
flat_image.size()#28×28等于784

torch.Size([3, 784])

#### nn.linear
通过权重和偏置，将数据线性组合。即$$xw+b$$既可降维，也可升维，取决于参数的维度。
#### nn.ReLU
线性修正单元作为非线性激活函数，在输入数据和输出数据之间建立复杂的映射关系。需传入经过linear层的数据，可帮助神经网络习得多种多样的“非寻常特征”。
#### nn.Sequential
将Sequential视为一个容器，里面放置了各种层。
#### nn.Softmax
将损失值转化为[0,1]上的取值。

## 四、最优化模型参数

In [23]:
lr=1e-3
batch_size=64
epochs=5

完成超参数的定义后，我们应使用最优化循环（optimization loop）来训练并最优化所设置的模型。最优化循环的单次迭代被称为epoch，包含：
1. 训练循环：利用数据集对参数迭代求解，并不断逼近最优参数；
2. 验证循环：也称测试循环，利用测试集数据，检验模型的预测效果是否得到改善。

最优化被分解为以下三个步骤：
1. 调用optimizer.zero_grad()函数来重新设置所有参数的梯度向量为0。这是因为梯度向量默认累加，需防止每次迭代后梯度向量的重复求和。
2. 调用loss.backward()函数来反向传播损失值，并存储损失值关于每个参数的梯度向量；
3. 在获得所有参数的梯度向量后，调用optimizer.step()函数进行梯度下降。

In [24]:
def train_loop(dataloader,model,loss_fn,optimizer):#分批训练
    size=len(dataloader.dataset)
    for batch,(X,y) in enumerate(dataloader):#该循环共执行（样本总数/batch_size）次
        pred=model(X)
        loss=loss_fn(pred,y)
        
        optimizer.zero_grad()#最优化步骤1
        loss.backward()#最优化步骤2
        optimizer.step()#最优化步骤3

        if batch % 100 == 0:
            loss,current=loss.item(),batch*len(y)#也可以使用len(X)，二者长度均为batch_size
            print(f'loss:{loss}   [{current}/{size}]')

def test_loop(dataloader,model,loss_fn):
    test_loss,correct=0,0
    
    with torch.no_grad():
        for X,y in dataloader:#该循环共执行（样本总数/batch_size）次
            pred=model(X)#X包含batch_size个数据
            test_loss += loss_fn(pred,y).item()#每执行一次，表示将batch_size个数据的损失值加总
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()#每执行一次，表示计算batch_size个数据的正确预测个数
    test_loss/=len(dataloader)#表示每batch_size个数据的loss，len(dataloader)返回的是所取的batch个数，本案例中为157，乘64之后为10048，近似等于10000
    correct/=len(dataloader.dataset)
    print(f'Test Error:\nAccuracy:{100*correct}%, Avg loss:{test_loss}\n')

In [25]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr)

lr=1e-3
batch_size=64
epochs=5

for i in range(epochs):
    print(f'Epoch {i+1}\n-----------')
    train_loop(train_dataloader,model,loss_fn,optimizer)
    test_loop(test_dataloader,model,loss_fn)
print('Done!')

Epoch 1
-----------
loss:2.29296612739563   [0/60000]
loss:2.292689323425293   [6400/60000]
loss:2.2929325103759766   [12800/60000]
loss:2.2711169719696045   [19200/60000]
loss:2.250555992126465   [25600/60000]
loss:2.2411553859710693   [32000/60000]
loss:2.2239990234375   [38400/60000]
loss:2.2021379470825195   [44800/60000]
loss:2.1875569820404053   [51200/60000]
loss:2.180978298187256   [57600/60000]
Test Error:
Accuracy:42.74%, Avg loss:2.1713928356292143

Epoch 2
-----------
loss:2.154694080352783   [0/60000]
loss:2.1570074558258057   [6400/60000]
loss:2.1610617637634277   [12800/60000]
loss:2.1124253273010254   [19200/60000]
loss:2.0882363319396973   [25600/60000]
loss:2.050867795944214   [32000/60000]
loss:2.0427889823913574   [38400/60000]
loss:1.9929804801940918   [44800/60000]
loss:1.9558532238006592   [51200/60000]
loss:1.961291790008545   [57600/60000]
Test Error:
Accuracy:58.08%, Avg loss:1.9254147376224493

Epoch 3
-----------
loss:1.9166574478149414   [0/60000]
loss:1.88